# Adding Back the Tides
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [2]:
start_day, end_day = 1, 30
days = range(start_day, end_day+1)

In [3]:
drop_vars = (
    "bounds_lon", "bounds_lat", "area", "deptht_bounds", "PAR",
    "time_centered_bounds", "time_counter_bounds", "dissolved_oxygen",
    "sigma_theta", "Fraser_tracer", "dissolved_inorganic_carbon", "total_alkalnity",
)

path = Path("/results2/SalishSea/nowcast-green.201905/")
files = [sorted(path.glob("{:02}mar19/SalishSea_1h_*_carp_T.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
e3t = mydata['e3t']

In [4]:
drop_vars = (
    "bounds_lon", "bounds_lat", "area", "depthu_bounds", 
    "time_centered_bounds", "time_counter_bounds",
)

path = Path("/results2/SalishSea/nowcast-green.201905/")
files = [sorted(path.glob("{:02}mar19/SalishSea_1h_*_grid_U.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u = mydata['vozocrtx']

In [ ]:
# might need to convert to e3u in this step

In [ ]:
ut_h = np.zeros((len(u.time_counter), len(u.y), len(u.x)))

for t in range(len(u.time_counter)):
    for y in range(len(u.y)):
        for x in range(len(u.x)):
            ut_h[t,y,x] = sum(u[t,:,y,x].values*e3t[t,:,y,x].values)/sum(e3t[t,:,y,x].values)

In [ ]:
#if my understanding is correct then we don't actually need the uc from the hourly files

# uc = np.zeros(np.shape(u))

# for t in range(len(u.time_counter)):
#     for z in range(40):
#         for y in range(len(u.y)):
#             for x in range(len(u.x)):
#                 uc[t,z,y,x] = u[t,z,y,x]-ut[t,y,x] 

In [ ]:
# Now get the required data from the daily files

drop_vars = (
    "bounds_lon", "bounds_lat", "area", "deptht_bounds", "PAR",
    "time_centered_bounds", "time_counter_bounds", "dissolved_oxygen",
    "sigma_theta", "Fraser_tracer", "dissolved_inorganic_carbon", "total_alkalnity",
)

path = Path("/results2/SalishSea/nowcast-green.201905/")
files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_carp_T.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
e3t = mydata['e3t']m

drop_vars = (
    "bounds_lon", "bounds_lat", "area", "depthu_bounds", 
    "time_centered_bounds", "time_counter_bounds",
)

path = Path("/results2/SalishSea/nowcast-green.201905/")
files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_grid_U.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u = mydata['vozocrtx']

In [ ]:
# again, might need to convert to e3u in this step

In [ ]:
ut_d = np.zeros((len(u.time_counter), len(u.y), len(u.x)))

for t in range(len(u.time_counter)):
    for y in range(len(u.y)):
        for x in range(len(u.x)):
            ut_d[t,y,x] = sum(u[t,:,y,x].values*e3t[t,:,y,x].values)/sum(e3t[t,:,y,x].values)

In [ ]:
uc_d = np.zeros(np.shape(u))

for t in range(len(u.time_counter)):
    for z in range(40):
        for y in range(len(u.y)):
            for x in range(len(u.x)):
                uc_d[t,z,y,x] = u[t,z,y,x]-ut_d[t,y,x]

In [ ]:
u_new = np.zeros(len(ut_h[:,0,0]),40,len(u.y), len(u.x))

for t in range(len(ut_h[:,0,0])):
    for z in range(40):
        for y in range(len(u.y)):
            for x in range(len(u.x)):
                u_new[t,z,y,x] = ut_h[t,y,x] + uc_d[t//24,z,y,x] 

In [ ]:
# and then i think that's it! rerun ariane with new currents (of course need to also do for v component)